In [28]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BufferedQueryInterface,QueryTableComponent,BokehTimeseriesGraphic,BokehButton

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div
from datetime import datetime
import pandas_datareader as pdr

# Bokeh Components
In this notebook we will outline how to use one of the most basic Bokeh Components, the button. Other notebooks show more exciting features, and are each designed to be independent. Bokeh Componets is an open source library that allows python developers to create interactive we applications with Bokeh more quickly. It was born out of frustration fighting with analytics libraries, and out of a need to share our analytics process internally. We are happy to share these time saving object oriented libraries with the world.

### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable. Having said that, we have developed a suite of unit tests which we we commit to passing, at least internally.

## The Datasoruce
This data source will consolidate all data access for the whole dashboard.



In [22]:
class ExampleComplexQuery(BufferedQueryInterface):
    '''
    An example of a data query that supports complex queries.
    '''
    
    #### Interface below -- Should be overriden by data sources
    #
    #
    #
    #
    def load_data_buffer(self):
        start_init=self._settings['start_date']
        end_init=self._settings['end_date']     

        # Grab data
        start=start_init
        end=end_init 

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]
        
        self.data = {
            'open_list':open_list,
            'close_list':close_list,
            'date_time':date_time,
            }

# All Controls
Next We create some controls to visualize the data.

In [29]:
class HelloButton(BokehButton):
    def handle_click(self,event):
        print("Hello I am a button!" + self._settings['name'])

### The Tabs
The most basic analytics applications you will create will likely be broken into tabs. In the folliwng example we illustrate how to create a couple of tabs

In [33]:
class BasicDashboard(Dashboard):
    def getLayout(self):
        # Lets create a dashboard that shows raw data on one tab, and visuals of that data on the other!
        # 
        self.complexQuery = ExampleComplexQuery({'start_date':datetime(2017,4,26),
                                       'end_date':datetime(2017,5,26)})
        settings = {'buffered_query_interface':self.complexQuery,
           'id_field':'open_list',
           'width':600,
           'height':300}

        self.table = QueryTableComponent(settings)
        panel1 = Panel(child=layout([ Div(text="This is the raw data", width=200, height=None),
                                    self.table.getBokehComponent()], sizing_mode='fixed'),title="Raw Data")
        
        # Lets create a second panel that is focused on the visuals
        settings = {'title':'Analytics Plot',
                          'width':600,
                          'height':300,
                          'query':self.complexQuery}

        settings['data_defs'] = [
                    {'y':'open_list','x':'date_time','key':'open','label':'Open','color':'green' },
                    {'y':'close_list','x':'date_time','key':'close','label':'Close','color':'yellow' }                    
                    ]
        
        self.graph = BokehTimeseriesGraphic(settings)        
        self.appendButton = HelloButton({'label':'Hello','name':'After this, we will change data'})
        panel2 = Panel(child=layout([ Div(text="This is the graph", width=200, height=None),self.graph.getBokehComponent()], sizing_mode='fixed'),title="Graph")
        
        # Include panels as tabs, with a control button:        
        tabs = Tabs(tabs=[panel2,panel1], width=700)        
        l = layout([tabs,Div(text="This is below ", width=200, height=None),self.appendButton.getBokehComponent()],sizing_mode='fixed')         
        return l
d = BasicDashboard()
d.showDashboard()

Loading BokehJS ...

Hello I am a button!After this, we will change data
Hello I am a button!After this, we will change data
Hello I am a button!After this, we will change data
Hello I am a button!After this, we will change data
Hello I am a button!After this, we will change data
